In [2]:
import os
import json
import pandas as pd
import pysrt
import re

In [6]:
def compute_iou(interval_1, interval_2):
    start_i, end_i = float(interval_1[0]), float(interval_1[1])
    start, end = float(interval_2[0]), float(interval_2[1])
    intersection = max(0, min(end, end_i) - max(start, start_i))
    union = min(max(end, end_i) - min(start, start_i), end-start + end_i-start_i)
    iou = float(intersection) / (union + 1e-8)
    return iou

In [26]:
def save_json(content, save_path):
    with open(save_path, 'w') as f:
        f.write(json.dumps(content))
def load_jsonl(filename):
    with open(filename, "r") as f:
        return [json.loads(l.strip("\n")) for l in f.readlines()]
def load_result_json(filename):
    with open(filename, "r") as f:
        data = json.load(f)
    #print(data)
    return data

BASELINE Evaluation

In [27]:
folders = ['baseline', 'visual_med', 'visual_nonmed', 'visual_med_with_audio']

In [28]:
root = "/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa_modified_script_sampled_exp"

In [29]:
for folder in folders:
    vid_json_folder = os.path.join(root, folder)

    print(f"{folder}\n")

    med_test_result_json_path = f"{vid_json_folder}/medical_test/all_data_test_moment_retrieval_BEST_with_audio.json"
    nonmed_test_result_json_path = f"{vid_json_folder}/nonmedical_test/all_data_test_moment_retrieval_BEST_with_audio.json"
    med_val_result_json_path = f"{vid_json_folder}/medical_val/all_data_val_moment_retrieval_BEST_with_audio.json"
    nonmed_val_result_json_path = f"{vid_json_folder}/nonmedical_val/all_data_val_moment_retrieval_BEST_with_audio.json"

    val = load_jsonl(f'/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa/all_data_val.json')
    test = load_jsonl(f'/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa/all_data_test.json')

    med_test_result = load_result_json(med_test_result_json_path)
    nonmed_test_result = load_result_json(nonmed_test_result_json_path)
    med_val_result = load_result_json(med_val_result_json_path)
    nonmed_val_result = load_result_json(nonmed_val_result_json_path)

    med_iou_thresh_3, med_iou_thresh_5, med_iou_thresh_7, cnt = 0, 0, 0, 0
    for key, value in test[0].items():
        if key in med_test_result:
            #print(key)
            vid = next(iter(value))
            vid2 = vid
            if vid not in med_test_result[key]:
                if list(med_test_result[key].keys())[0].startswith(vid.replace(".mp4", "")):
                    vid2 = list(med_test_result[key].keys())[0]
                else:
                    print(vid, list(med_test_result[key].keys())[0])
                    continue
            iou = compute_iou(value[vid]['bounds'], med_test_result[key][vid2]['bounds'])
            cnt += 1
            if iou >= 0.3:
                med_iou_thresh_3 += 1 
            if iou >= 0.5:
                med_iou_thresh_5 += 1 
            if iou >= 0.7:
                med_iou_thresh_7 += 1 
    print(f"Med TEST IoU=0.3: {med_iou_thresh_3/(cnt) * 100}\nIoU=0.5: {med_iou_thresh_5/(cnt) * 100}\nIoU=0.7: {med_iou_thresh_7/(cnt) * 100}\ntotal queries:{cnt}")

    nonmed_iou_thresh_3, nonmed_iou_thresh_5, nonmed_iou_thresh_7, cnt = 0, 0, 0, 0
    for key, value in test[0].items():
        if key in nonmed_test_result:
            vid = next(iter(value))
            vid2 = vid
            if vid not in nonmed_test_result[key]:
                if list(nonmed_test_result[key].keys())[0].startswith(vid.replace(".mp4", "")):
                    vid2 = list(nonmed_test_result[key].keys())[0]
                else:
                    print(vid, list(nonmed_test_result[key].keys())[0])
                    continue
            iou = compute_iou(value[vid]['bounds'], nonmed_test_result[key][vid2]['bounds'])
            cnt += 1
            if iou >= 0.3:
                nonmed_iou_thresh_3 += 1 
            if iou >= 0.5:
                nonmed_iou_thresh_5 += 1 
            if iou >= 0.7:
                nonmed_iou_thresh_7 += 1 
    print(f"\nNonmed TEST IoU=0.3: {nonmed_iou_thresh_3/(cnt) * 100}\nIoU=0.5: {nonmed_iou_thresh_5/(cnt) * 100}\nIoU=0.7: {nonmed_iou_thresh_7/(cnt) * 100}\ntotal queries:{cnt}")


    med_iou_thresh_3, med_iou_thresh_5, med_iou_thresh_7, cnt = 0, 0, 0, 0
    for key, value in val[0].items():
        if key in med_val_result:
            vid = next(iter(value))
            vid2 = vid
            if vid not in med_val_result[key]:
                if list(med_val_result[key].keys())[0].startswith(vid.replace(".mp4", "")):
                    vid2 = list(med_val_result[key].keys())[0]
                else:
                    print(vid, list(med_val_result[key].keys())[0])
                    continue
            iou = compute_iou(value[vid]['bounds'], med_val_result[key][vid2]['bounds'])
            cnt += 1
            if iou >= 0.3:
                med_iou_thresh_3 += 1 
            if iou >= 0.5:
                med_iou_thresh_5 += 1 
            if iou >= 0.7:
                med_iou_thresh_7 += 1 
    print(f"\nMed VAL IoU=0.3: {med_iou_thresh_3/(cnt) * 100}\nIoU=0.5: {med_iou_thresh_5/(cnt) * 100}\nIoU=0.7: {med_iou_thresh_7/(cnt) * 100}\ntotal queries:{cnt}")

    nonmed_iou_thresh_3, nonmed_iou_thresh_5, nonmed_iou_thresh_7, cnt = 0, 0, 0, 0
    for key, value in val[0].items():
        if key in nonmed_val_result:
            vid = next(iter(value))
            vid2 = vid
            if vid not in nonmed_val_result[key]:
                if list(nonmed_val_result[key].keys())[0].startswith(vid.replace(".mp4", "")):
                    vid2 = list(nonmed_val_result[key].keys())[0]
                else:
                    print(vid, list(nonmed_val_result[key].keys())[0])
                    continue
            iou = compute_iou(value[vid]['bounds'], nonmed_val_result[key][vid2]['bounds'])
            cnt += 1
            if iou >= 0.3:
                nonmed_iou_thresh_3 += 1 
            if iou >= 0.5:
                nonmed_iou_thresh_5 += 1 
            if iou >= 0.7:
                nonmed_iou_thresh_7 += 1 
    print(f"\nNonmed VAL IoU=0.3: {nonmed_iou_thresh_3/(cnt) * 100}\nIoU=0.5: {nonmed_iou_thresh_5/(cnt) * 100}\nIoU=0.7: {nonmed_iou_thresh_7/(cnt) * 100}\ntotal queries:{cnt}")

    print("\n\n")

baseline

Med TEST IoU=0.3: 4.5
IoU=0.5: 2.5
IoU=0.7: 2.0
total queries:200

Nonmed TEST IoU=0.3: 7.000000000000001
IoU=0.5: 4.5
IoU=0.7: 2.0
total queries:200

Med VAL IoU=0.3: 8.5
IoU=0.5: 3.5000000000000004
IoU=0.7: 1.0
total queries:200

Nonmed VAL IoU=0.3: 15.0
IoU=0.5: 4.0
IoU=0.7: 2.5
total queries:200



visual_med

Med TEST IoU=0.3: 9.0
IoU=0.5: 3.0
IoU=0.7: 1.0
total queries:200

Nonmed TEST IoU=0.3: 14.000000000000002
IoU=0.5: 3.5000000000000004
IoU=0.7: 0.5
total queries:200

Med VAL IoU=0.3: 10.5
IoU=0.5: 4.5
IoU=0.7: 2.0
total queries:200

Nonmed VAL IoU=0.3: 7.000000000000001
IoU=0.5: 4.0
IoU=0.7: 1.0
total queries:200



visual_nonmed

Med TEST IoU=0.3: 9.0
IoU=0.5: 5.0
IoU=0.7: 1.0
total queries:200

Nonmed TEST IoU=0.3: 9.5
IoU=0.5: 3.5000000000000004
IoU=0.7: 0.5
total queries:200

Med VAL IoU=0.3: 7.000000000000001
IoU=0.5: 2.5
IoU=0.7: 1.0
total queries:200

Nonmed VAL IoU=0.3: 4.5
IoU=0.5: 2.5
IoU=0.7: 0.5
total queries:200



visual_med_with_audio

Med TEST IoU=0